In [2]:
import torch
%load_ext autoreload
%autoreload 2

In [3]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [7]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class='lamp')
val_dataset = ShapeNet('val', filter_class='lamp')
test_dataset = ShapeNet('test', filter_class='lamp')

print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
print(f'Length of val set: {len(val_dataset)}')  # expected output: 153540
print(f'Length of test set: {len(test_dataset)}')  # expected output: 64

Length of train set: 1854
Length of val set: 232
Length of test set: 232


In [12]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = test_dataset[231]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)
print(f'Target DF: {type(sample["target_df"])}')  # expected output: <class 'numpy.ndarray'>

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)
Target DF: <class 'numpy.ndarray'>


Output()

In [6]:
##################
#                #
#    TRAINING    #
#                #
##################

In [11]:
# OVERFIT
from scripts import train
config = {
    'experiment_name': 'overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 100,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None,
    'filter_class': 'car_airplane',
    'decoder_var' : True

}
train.main(config)

Using device: cuda:0
Data length 9994
Training params: 3
[000/00099] train_loss: 0.288746 kl_loss: 0.166894 normal_loss: 0.121852
[000/00199] train_loss: 0.225641 kl_loss: 0.166353 normal_loss: 0.059288
[000/00299] train_loss: 0.219455 kl_loss: 0.166727 normal_loss: 0.052728
[001/00086] train_loss: 0.176847 kl_loss: 0.128196 normal_loss: 0.048651
[001/00186] train_loss: 0.170828 kl_loss: 0.122256 normal_loss: 0.048573
[001/00286] train_loss: 0.169678 kl_loss: 0.122033 normal_loss: 0.047645
[002/00073] train_loss: 0.142393 kl_loss: 0.094651 normal_loss: 0.047742
[002/00173] train_loss: 0.130873 kl_loss: 0.084667 normal_loss: 0.046206


KeyboardInterrupt: 

In [8]:
# AIRPLANE VAD
from scripts import train
config = {
    'experiment_name': 'airplane_vad_final',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 4045
Training params: 3
[001/00035] train_loss: 0.103237 kl_loss: 0.163130 normal_loss: 0.098343
[003/00007] train_loss: 0.048654 kl_loss: 0.136225 normal_loss: 0.044567
[004/00043] train_loss: 0.040166 kl_loss: 0.099543 normal_loss: 0.037180
[006/00015] train_loss: 0.036385 kl_loss: 0.077942 normal_loss: 0.034047
[007/00051] train_loss: 0.034134 kl_loss: 0.067799 normal_loss: 0.032100
[009/00023] train_loss: 0.033282 kl_loss: 0.061263 normal_loss: 0.031444
[010/00059] train_loss: 0.032224 kl_loss: 0.054992 normal_loss: 0.030574
[012/00031] train_loss: 0.031978 kl_loss: 0.048546 normal_loss: 0.030522
[014/00003] train_loss: 0.030175 kl_loss: 0.044901 normal_loss: 0.028828
[015/00039] train_loss: 0.029017 kl_loss: 0.042187 normal_loss: 0.027751
[017/00011] train_loss: 0.028610 kl_loss: 0.041694 normal_loss: 0.027360
[018/00047] train_loss: 0.027759 kl_loss: 0.043558 normal_loss: 0.026452
[020/00019] train_loss: 0.025344 kl_loss: 0.044607 normal_loss: 0.0

KeyboardInterrupt: 

In [ ]:
# AIRPLANE AD
from scripts import train
config = {
    'experiment_name': 'airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# CHAIR
from scripts import train
config = {
    'experiment_name': 'chair_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# SOFA & CHAIR VAD
from scripts import train
config = {
    'experiment_name': 'sofa_chair_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa_chair',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# SOFA VAD
from scripts import train
config = {
    'experiment_name': 'sofa_vad_0_03kl',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# SOFA AD
from scripts import train
config = {
    'experiment_name': 'sofa_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# SOFA VAD
from scripts import train
config = {
    'experiment_name': 'car_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# CAR AD
from scripts import train
config = {
    'experiment_name': 'car_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# CAR AIRPLANE AD
from scripts import train
config = {
    'experiment_name': 'car_airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': 'car_airplane_ad',
    'filter_class': 'car_airplane',
    'decoder_var' : False
}
train.main(config)

In [13]:
# TABLE VAD
from scripts import train
config = {
    'experiment_name': 'table_vad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'table',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 4800
Training params: 3
[001/00024] train_loss: 0.169897 kl_loss: 0.163592 normal_loss: 0.164990
[002/00049] train_loss: 0.122329 kl_loss: 0.142284 normal_loss: 0.118060
[003/00074] train_loss: 0.115421 kl_loss: 0.120233 normal_loss: 0.111814
[005/00024] train_loss: 0.109729 kl_loss: 0.118276 normal_loss: 0.106181
[006/00049] train_loss: 0.106850 kl_loss: 0.121355 normal_loss: 0.103209
[007/00074] train_loss: 0.102989 kl_loss: 0.127613 normal_loss: 0.099161
[009/00024] train_loss: 0.102100 kl_loss: 0.138114 normal_loss: 0.097956
[010/00049] train_loss: 0.100053 kl_loss: 0.145995 normal_loss: 0.095673
[011/00074] train_loss: 0.097527 kl_loss: 0.151523 normal_loss: 0.092981
[013/00024] train_loss: 0.093813 kl_loss: 0.154968 normal_loss: 0.089164
[014/00049] train_loss: 0.092380 kl_loss: 0.159106 normal_loss: 0.087606
[015/00074] train_loss: 0.092439 kl_loss: 0.161559 normal_loss: 0.087593
[017/00024] train_loss: 0.090308 kl_loss: 0.163313 normal_loss: 0.0

In [14]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [15]:
from scripts.visualize import visualize_dataset_sample, visualize_ad, visualize_vad, visualize_vad_norm, visualize_vad_norm

In [77]:
import random
experiment = "table_vad"
experiment2 = "sofa_ad"
# experiment2 = "sofa_ad"
filter_class = "table"
index = 4123
index1 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
index2 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
a1 = 0.5
a2 = 1 - a1
#-------
# visualize_vad_norm(experiment2)
visualize_vad_norm(experiment)
visualize_ad(experiment, index1)
# visualize_vad_norm(experiment2)
# visualize_ad(experiment, index)
#-------
# visualize_vad_norm(experiment)
# visualize_vad_norm(experiment2)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad(experiment, index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

Output()

In [6]:
####################
#                  #
#    EVALUATION    #
#                  #
####################

In [96]:
from scripts.evaluate import mmd, generate_samples, convert_df_to_point_cloud, visualize_point_cloud, chamfer_distance_numpy, mmd
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import numpy as np

In [100]:
# MMD

# generate n new samples
n = 2
samples = generate_samples('sofa_vad', n)

sample1 = samples[0]
sample2 = samples[1]

# visualize
input_mesh = marching_cubes(sample1.squeeze(0).detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

input_mesh = marching_cubes(sample2.squeeze(0).detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

point_cloud = convert_df_to_point_cloud(sample1.squeeze(0))
visualize_point_cloud(point_cloud)

point_cloud2 = convert_df_to_point_cloud(sample2.squeeze(0))
visualize_point_cloud(point_cloud2)

Output()

Output()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [101]:
# calculate chamfer distance between the 2 samples
chamfer_distance_numpy(point_cloud, point_cloud2)

1.9491640607287115

In [102]:
# calculate mmd between the 2 samples
# Note: the final value changes a bit every time because every time a new point cloud is generated
# Note: mmd function takes a lot of time, needs optimization
mmd(samples, samples)

array([1.21344659, 1.21994092])